# Introduction

Our goal is to determine the most superior Pittsburgh neighborhood by using data provided by the WPDC. and a cruical factor to determining which neighborhood is the best is if its visitor friendly. Attracting tourists/visitors especially, is very beneficial to the overall economy of a city.

![travel](https://media.tenor.com/kMva7jHp2ngAAAAM/national-camera-day-donald-duck.gif)

# The Metric

In order to determine whats the best neighborhood for visitors to Pittsburgh we looked at:
* Locations of public art pieces
* 
* 

### Part 1 - Public Art Pieces

### Part 2 -

### Part 3 -

# The Best Neighborhood

# Conclusion